In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.measure as skm
from Outils.dataloader import load_CIFAR10

In [2]:
# Load the raw CIFAR-10 data.
cifar10_dir = 'Dataset/cifar-10-batches-py'

# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
X = np.concatenate([X_train, X_test])
X.shape

(60000, 32, 32, 3)

In [40]:
def channel_encoder(channel, min, max, m = 50):
    out = np.copy(channel)
    level_im = np.zeros(channel.shape)
    H, W = channel.shape
    level_im[0,:] = m*(channel[0,:] - min[0,:])/(max[0,:] - min[0,:])
    level_im[:,0] = m*(channel[:,0] - min[:,0])/(max[:,0] - min[:,0])
    level_im = level_im.astype(int)

    for w in range(1, W):
        for h in range(1, H):
            level = np.floor((level_im[h-1,w] + level_im[h,w-1] + level_im[h-1,w-1])/3)
            prediction = min[h,w] + np.floor(level*(max[h,w] - min[h,w])/m)
            level_im[h,w] = level
            out[h,w] = channel[h,w] - prediction

    return out


In [41]:
def channel_decoder(channel, min, max, m = 50):
    out = np.copy(channel)
    H, W = channel.shape
    level_im = np.zeros(channel.shape)
    level_im[0,:] = m*(channel[0,:] - min[0,:])/(max[0,:] - min[0,:])
    level_im[:,0] = m*(channel[:,0] - min[:,0])/(max[:,0] - min[:,0])
    level_im = level_im.astype(int)

    for w in range(1, W):
        for h in range(1, H):
            level = np.floor((level_im[h-1,w] + level_im[h,w-1] + level_im[h-1,w-1])/3)
            prediction = min[h,w] + np.floor(level*(max[h,w] - min[h,w])/m)
            level_im[h,w] = level
            out[h,w] = channel[h,w] + prediction
    return out

In [42]:
min = np.min(X, axis=0)
max = np.max(X, axis=0) 

In [43]:
Xe = channel_encoder(X[0,:,:,0], min[:,:,0], max[:,:,0])

In [44]:
Xd = channel_decoder(Xe, min[:,:,0], max[:,:,0])

In [45]:
(Xd != X[0,:,:,0]).sum()

0

In [47]:
def Encoder(X, m = 50):
    '''
    Min Max Predictive without storing levels
    '''
    min = np.min(X, axis=0)
    max = np.max(X, axis=0)
    N = X.shape[0]
    Y = np.zeros(X.shape)

    for n in range(N):
        for c in range(3):
            Y[n,:,:,c] = channel_encoder(X[n,:,:,c], min[:,:,c], max[:,:,c], m)
    
    return Y, min, max


In [48]:
def Decoder(Y, min, max, m = 50):
    '''
    Min Max Predictive without storing levels
    '''
    N = Y.shape[0]
    X = np.zeros(Y.shape)

    for n in range(N):
        for c in range(3):
            X[n,:,:,c] = channel_decoder(Y[n,:,:,c], min[:,:,c], max[:,:,c], m)
    
    return X


In [53]:
Xe, min, max = Encoder(X[1:20])
Xd = Decoder(X, min, max) 

In [50]:
(Xd != X[1:20]).sum()

167493393

In [ ]:
def Encoder2(X, m = 50):
    '''
    Min Max Predictive without storing levels
    '''
    min = np.min(X, axis=0)
    max = np.max(X, axis=0)
    _, H, W, _ = X.shape
    Y = np.copy(X)
    level_im = np.zeros(X.shape)
    level_im[:,0,:,:] = m*(X[:,0,:,:] - min[0,:,:])/(max[0,:,:] - min[0,:,:])
    level_im[:,:,0,:] = m*(X[:,:,0,:] - min[:,0,:])/(max[:,0,:] - min[:,0,:])
    level_im = level_im.astype(int)

    for w in range(1, W):
        for h in range(1, H):
            level = np.floor((level_im[:,h-1,w,:] + level_im[:,h,w-1,:] + level_im[:,h-1,w-1,:])/3)  
            prediction = min[h,w,:] + np.floor(level*(max[h,w,:] - min[h,w,:])/m)
            level_im[:,h,w,:] = level
            Y[:,h,w,:] = X[:,h,w,:] - prediction 
    
    return Y, min, max

In [ ]:
def Decoder2(Y, min, max, m = 50):
    '''
    Min Max Predictive without storing levels
    '''
    _, H, W, _ = Y.shape
    X = np.copy(Y)

    level_im = np.zeros(Y.shape)
    level_im[:,0,:,:] = m*(Y[:,0,:,:] - min[0,:,:])/(max[0,:,:] - min[0,:,:])
    level_im[:,:,0,:] = m*(Y[:,:,0,:] - min[:,0,:])/(max[:,0,:] - min[:,0,:])
    level_im = level_im.astype(int)

    for w in range(1, W):
        for h in range(1, H):
            level = np.floor((level_im[:,h-1,w,:] + level_im[:,h,w-1,:] + level_im[:,h-1,w-1,:])/3)  
            prediction = min[h,w,:] + np.floor(level*(max[h,w,:] - min[h,w,:])/m)
            level_im[:,h,w,:] = level
            X[:,h,w,:] = Y[:,h,w,:] + prediction 
    
    return X